In [10]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import seaborn
from matplotlib import pyplot
%matplotlib inline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\idris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\idris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\idris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\idris\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [11]:
hotel = pd.read_csv("hotels_in_ethiopia_data.csv")
restaurant = pd.read_csv("restaurants_in_ethiopia.csv")
attraction = pd.read_csv("attractions_in_ethiopia_edited_data.csv")

# DATA CLEANING AND FEATURE ENGINEERING

In [12]:
del restaurant['email']
del restaurant['phone']
del attraction['Image_url']
del attraction['Offer_price']
del attraction['Offer_primary_category']


In [13]:
hotel = hotel.dropna()
restaurant.head()

,Restaurant,original_location,tags,opening_hours,reviews_distribution,ratings_distribution,popularity,awards,claimed,address,...,meals,rating,review_count,website,longitude,latitude,price_range,about,url,timestamp
0,The Kitchen,"['Africa', 'Ethiopia', 'Addis Ababa', 'Addis A...","['Mediterranean', 'Healthy', 'Middle Eastern']","['Sun 6:00 AM - 10:30 PM', 'Mon 6:00 AM - 10:3...","{'Excellent': 157, 'Very good': 13, 'Average':...","{'Food': 4.5, 'Service': 5.0, 'Value': 4.5}",#1of 269Restaurants in Addis Ababa,Travelers' Choice 2023,Claimed,"Meskel Square, Addis Ababa 2150 Ethiopia",...,NaN,5.0,177.0,http://www.hyattrestaurants.com/en/dining/ethi...,9.010573,38.763622,€9 - €11,With 100+ seats it is the largest restaurant w...,https://www.tripadvisor.com/Restaurant_Review-...,2023-10-23T15:27:51Z
1,Dok Restaurant,"['Africa', 'Ethiopia', 'Addis Ababa', 'Addis A...","['Italian', 'Pizza', 'Mediterranean']","['Sun 9:00 AM - 10:00 PM', 'Mon 9:00 AM - 11:0...","{'Excellent': 34, 'Very good': 20, 'Average': ...","{'Food': 4.0, 'Service': 4.5, 'Value': 4.0}",#30of 269Restaurants in Addis Ababa,NaN,Claimed,"Cameroon Street, Addis Ababa 2366 Ethiopia",...,NaN,4.5,77.0,NaN,8.99411,38.790360,€6 - €11,Dok restaurant offers its customers good food....,https://www.tripadvisor.com/Restaurant_Review-...,2023-10-23T15:28:14Z
2,Opium Restaurant,"['Africa', 'Ethiopia', 'Addis Ababa', 'Addis A...","['Fast Food', 'Vegetarian Friendly', 'Vegan Op...","['Sun 7:30 AM - 11:30 PM', 'Mon 7:30 AM - 11:3...","{'Excellent': 25, 'Very good': 4, 'Average': 2...","{'Food': 4.5, 'Service': 4.5, 'Value': 4.0}",#29of 269Restaurants in Addis Ababa,NaN,Claimed,"Cape Verde st Bole Atlas, Rakan Building, Grou...",...,NaN,5.0,32.0,http://www.opiumaddis.com,8.963198,38.778145,PLN 8 - PLN 14,NaN,https://www.tripadvisor.com/Restaurant_Review-...,2023-10-23T15:28:34Z
3,Fendika Azmari Bet,"['Africa', 'Ethiopia', 'Addis Ababa', 'Addis A...","['African', 'Ethiopian', 'Vegetarian Friendly']",[],"{'Excellent': 19, 'Very good': 9, 'Average': 1...","{'Food': 4.0, 'Service': 4.5, 'Value': 4.5}",#9of 83African in Addis Ababa#27of 269Restaura...,NaN,NaN,"Zewditu Street, Addis Ababa Ethiopia",...,Dinner,4.5,33.0,NaN,9.019039,38.770240,NaN,NaN,https://www.tripadvisor.com/Restaurant_Review-...,2023-10-23T15:29:06Z
4,Villaverde Addis Ababa,"['Africa', 'Ethiopia', 'Addis Ababa', 'Addis A...","['Italian', 'Pizza', 'Mediterranean']","['Sun 10:00 AM - 2:00 AM', 'Mon 10:00 AM - 2:0...","{'Excellent': 28, 'Very good': 8, 'Average': 2...","{'Food': 4.5, 'Service': 4.5, 'Value': 4.5}",#8of 43European in Addis Ababa#26of 269Restaur...,NaN,Claimed,"Meskel Flower Behind Saint Yared Church, Addis...",...,NaN,4.5,57.0,http://www.villaverdeaddisababa.com,8.994789,38.766570,₪61 - ₪203,VILLAVERDE is a chic cultural hub where outsta...,https://www.tripadvisor.com/Restaurant_Review-...,2023-10-23T15:29:24Z


In [5]:
# Merge the hotel and restaurant DataFrames using the 'location' column
merged_data = pd.merge(hotel, attraction, on='Location', how='outer')
merged_data = merged_data.drop(['Star', 'Price Per Night', 'Amenities', 'Location_subcategory', 'Latitude',
       'Longitude', 'Description', 'Ranking_position',
       'Ranking_string', 'Rating', 'Raw_ranking', 'Attraction_subcategory',
       'Attraction_subtype', 'Website'], axis=1)

In [6]:
merged_data = merged_data[['Location', 'Hotel', 'Attraction', 'Activities']]
merged_data.head()

,Location,Hotel,Attraction,Activities
0,Addis Ababa,Capital Hotel And Spa,Holy Trinity Cathedral,Explore the Danakil Depression in 6 Days
1,Addis Ababa,Capital Hotel And Spa,Red Terror Martyrs Memorial Museum,Discover Red Terror Martyrs Memorial Museum
2,Addis Ababa,Capital Hotel And Spa,Mount Entoto,Mount Entoto Full Day Tour with Hotel Pickup a...
3,Addis Ababa,Capital Hotel And Spa,Adadi Maryam,Day trip to Adadi Mariyam and Melka kunture
4,Addis Ababa,Capital Hotel And Spa,National Museum of Ethiopia,Full Day Tour of National Museum of Ethiopia w...


In [7]:
#removing punctuations and changing to lower case
hotel['Amenities'] = hotel['Amenities'].str.replace(',', '')
hotel['Amenities'] = hotel['Amenities'].str.lower()

# CITY-BASED HOTEL RECOMMENDER SYSTEM

In [8]:
def citybased(Location):
    hotel['Location']=hotel['Location'].str.lower()
    citybase=hotel[hotel['Location']==Location.lower()]
    citybase=citybase.sort_values(by='Star',ascending=False)
    if(citybase.empty==0):
        hname=citybase[['Hotel', 'Location', 'Star', 'Price Per Night', 'Amenities']]
        return hname.head()
    else:
        print('No Hotels Available')
        
print('Top hotels around me')
citybased('mekele')

Top hotels around me


,Hotel,Location,Star,Price Per Night,Amenities
30,Romanat Hotel,mekele,5,"$2,580.00",restaurant free breakfast horseback ridi...
24,Atse Yohannes Hotel,mekele,4,"$2,260.00",restaurant free breakfast swimming...
25,Mekelle Hotel,mekele,4,"$1,713.00",restaurant free wifi ...
26,Desta International Hotel,mekele,4,"$2,399.00",restaurant free breakfast swimming...
27,Planet Hotel,mekele,4,"$3,874.00",restaurant free breakfast swimming...


# CITY-BASED RESTAURANT RECOMMENDER SYSTEM

In [21]:
def citybased(Location):
    restaurant['Location']=restaurant['Location'].str.lower()
    citybase=restaurant[restaurant['Location']==Location.lower()]
    citybase=citybase.sort_values(by='Star',ascending=False)
    if(citybase.empty==0):
        rname=citybase[['Restaurant', 'Location', 'Star']]
        return rname.head()
    else:
        print('No Restaurants Available')
        
print('Top restaurants around me')
citybased('addis ababa')

In [ ]:
url = "https://best-booking-com-hotel.p.rapidapi.com/booking/best-accommodation"
user_location = input("Enter the city name for hotel recommendations: ")  # Get user input for the location

querystring = {"cityName": user_location, "countryName": "Ethiopia"}


In [52]:
import requests

url = "https://best-booking-com-hotel.p.rapidapi.com/booking/best-accommodation"

querystring = {"cityName":"mekele","countryName":"Ethiopia"}

headers = {
	"X-RapidAPI-Key": "3d888e6586msh3821c11a4466d53p148a14jsna7ebedd6c86d",
	"X-RapidAPI-Host": "best-booking-com-hotel.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'name': 'Homeland Hotel Mekelle', 'link': 'https://www.booking.com/hotel/et/homeland-mekelle.de.html?aid=1938431', 'rating': 9.5}


In [53]:
import requests

# API endpoint and query parameters
url = "https://best-booking-com-hotel.p.rapidapi.com/booking/best-accommodation"
user_location = input("Enter the city name for hotel recommendations: ")  # Get user input for the location

querystring = {"cityName": user_location, "countryName": "Ethiopia"}

# Headers with your RapidAPI key
headers = {
    "X-RapidAPI-Key": "3d888e6586msh3821c11a4466d53p148a14jsna7ebedd6c86d",  # Replace with your actual RapidAPI key
    "X-RapidAPI-Host": "best-booking-com-hotel.p.rapidapi.com"
}

# Send a GET request to the hotel data API
response = requests.get(url, headers=headers, params=querystring)

if response.status_code == 200:
    hotel_data = response.json()
    
    if "hotels" in hotel_data:
        # Extract and display hotel recommendations
        for hotel in hotel_data["hotels"]:
            print(f"Hotel Name: {hotel.get('name', 'N/A')}")
            print(f"Location: {hotel.get('location', 'N/A')}")
            print(f"Rating: {hotel.get('rating', 'N/A')}")
            print(f"Amenities: {', '.join(hotel.get('amenities', []))}")
            print(f"Price per Night: {hotel.get('price_per_night', 'N/A')} USD")
            print(f"Image URL: {hotel.get('image_url', 'N/A')}")
            print("-" * 40)
    else:
        print("No hotel recommendations available for this location.")
else:
    print(f"Failed to fetch hotel recommendations. Error code: {response.status_code}")
    print(response.text)  # Print the response for debugging


Enter the city name for hotel recommendations: mekele
No hotel recommendations available for this location.


In [46]:
import requests

# API endpoint and query parameters
url = "https://best-booking-com-hotel.p.rapidapi.com/booking/best-accommodation"
querystring = {"cityName": "Addis Ababa", "countryName": "Ethiopia"}

# Headers with your RapidAPI key
headers = {
    "X-RapidAPI-Key": "3d888e6586msh3821c11a4466d53p148a14jsna7ebedd6c86d",  # Replace with your actual RapidAPI key
    "X-RapidAPI-Host": "best-booking-com-hotel.p.rapidapi.com"
}

# Define query parameters based on the user's input (not used in this context)
# You can add query parameters specific to the API if needed

# Send a GET request to the hotel data API
response = requests.get(url, headers=headers, params=querystring)

if response.status_code == 200:
    # Parse the JSON response from the API
    hotel_data = response.json()

    # Extract and display hotel recommendations
    for hotel in hotel_data['Hotel']:
        print(f"Hotel Name: {hotel['name']}")
        print(f"Location: {hotel['location']}")
        print(f"Rating: {hotel['rating']}")
        print(f"Amenities: {', '.join(hotel['amenities'])}")
        print(f"Price per Night: {hotel['price_per_night']} USD")
        print(f"Image URL: {hotel['image_url']}")
        print("-" * 40)
else:
    print('Failed to fetch hotel recommendations. Please try again later.')


KeyError: 'Hotel'